# Tổng kết và Hướng phát triển## Tóm tắt

### Tóm tắt lại hành trình1. **Bài toán:** Tối ưu hóa $L(W)$.2. **Thách thức:** Local Minima, Saddle Points, Vanishing Gradient.3. **Giải pháp cơ bản:** Dùng SGD $\nabla L$.4. **Hạn chế của SGD:** Vấn đề **Gradient Thưa**.5. **Giải pháp thích ứng:** Adagrad.

## Các hướng cải tiến (Future Work)Chỉ ra nhược điểm "dừng học sớm" của Adagrad chính là động lực cho các thuật toán tiếp theo.

### RMSPropGiải quyết vấn đề $s_t$ tăng mãi bằng cách sử dụng _trung bình trượt theo hàm mũ (exponential moving average)_.**Công thức:**$$s_t = \beta s_{t-1} + (1-\beta) g_t^2$$Thay vì tích lũy vô hạn như Adagrad ($s_t = s_{t-1} + g_t^2$), RMSProp chỉ "nhớ" lịch sử gần đây với tham số decay $\beta \in [0.9, 0.99]$.**Ưu điểm:**- Ngăn $s_t$ tăng lớn vô hạn- Learning rate không giảm quá nhanh- Phù hợp với non-stationary objectives**Update rule:**$$W_{t+1} = W_t - \frac{\eta}{\sqrt{s_t + \epsilon}} \odot g_t$$

### Adam (Adaptive Moment Estimation)Kết hợp ý tưởng của RMSProp và Momentum.**Công thức:****Moment bậc 1 (Momentum):**$$m_t = \beta_1 m_{t-1} + (1-\beta_1) g_t$$**Moment bậc 2 (RMSProp):**$$v_t = \beta_2 v_{t-1} + (1-\beta_2) g_t^2$$**Bias correction:**$$\hat{m}_t = \frac{m_t}{1-\beta_1^t}, \quad \hat{v}_t = \frac{v_t}{1-\beta_2^t}$$**Update rule:**$$W_{t+1} = W_t - \frac{\eta}{\sqrt{\hat{v}_t} + \epsilon} \hat{m}_t$$**Tham số mặc định:**- $\beta_1 = 0.9$ (momentum)- $\beta_2 = 0.999$ (RMSProp decay)- $\eta = 0.001$ (learning rate)- $\epsilon = 10^{-8}$**Ưu điểm:**- Kết hợp lợi ích của cả momentum và adaptive learning rate- Bias correction giúp ổn định ở bước đầu- Là optimizer phổ biến nhất trong deep learning hiện đại

## So sánh các thuật toán| Thuật toán | Learning Rate | Momentum | Adaptive LR | Complexity ||---|---|---|---|---|| **SGD** | Cố định | ✗ | ✗ | $O(1)$ || **SGD + Momentum** | Cố định | ✓ | ✗ | $O(1)$ || **Adagrad** | Adaptive | ✗ | ✓ | $O(d)$ || **RMSProp** | Adaptive | ✗ | ✓ (với decay) | $O(d)$ || **Adam** | Adaptive | ✓ | ✓ (với decay) | $O(d)$ |_Trong đó $d$ là số chiều của tham số._### Khi nào dùng thuật toán nào?- **SGD/SGD+Momentum:** Khi cần tốc độ nhanh, bộ nhớ thấp, hoặc muốn generalization tốt với learning rate schedule phù hợp.- **Adagrad:** Dữ liệu sparse, NLP với features hiếm, không cần training lâu.- **RMSProp:** Non-stationary objectives, RNN, hoặc khi Adagrad bị "đóng băng".- **Adam:** Lựa chọn mặc định cho hầu hết các tác vụ deep learning, đặc biệt khi không muốn tune learning rate nhiều.

In [ ]:
import torchimport torch.optim as optim# Giả sử có model# model = YourModel()# 1. SGDoptimizer_sgd = optim.SGD(model.parameters(), lr=0.01)# 2. SGD with Momentumoptimizer_sgd_momentum = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)# 3. Adagradoptimizer_adagrad = optim.Adagrad(model.parameters(), lr=0.01, eps=1e-8)# 4. RMSPropoptimizer_rmsprop = optim.RMSprop(model.parameters(), lr=0.001, alpha=0.99, eps=1e-8)# 5. Adam (Most popular)optimizer_adam = optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-8)# 6. AdamW (Adam with weight decay fix)optimizer_adamw = optim.AdamW(model.parameters(), lr=0.001, betas=(0.9, 0.999),                                eps=1e-8, weight_decay=0.01)print("Các optimizer đã được khởi tạo!")print("Lựa chọn phổ biến nhất: Adam hoặc AdamW")

## Kết luận### Hành trình từ SGD đến AdamQuá trình phát triển các thuật toán tối ưu trong deep learning là một chuỗi các cải tiến dần:1. **SGD** → Đơn giản nhưng khó điều chỉnh learning rate2. **SGD + Momentum** → Thêm "quán tính" giúp vượt qua local minima3. **Adagrad** → Adaptive learning rate per-parameter, tốt cho sparse data4. **RMSProp** → Khắc phục Adagrad bằng exponential decay5. **Adam** → Kết hợp momentum + RMSProp + bias correction### Bài học quan trọng- Không có optimizer "tốt nhất" cho mọi bài toán- Adam là lựa chọn an toàn cho hầu hết trường hợp- SGD + Momentum vẫn được ưa chuộng khi cần generalization tốt- Hiểu cơ chế hoạt động giúp debug và fine-tune hiệu quả hơn### Xu hướng hiện tại- **AdamW:** Cải thiện weight decay của Adam- **LAMB, LARS:** Cho large-batch training- **Lookahead, RAdam:** Cải thiện convergence và stability- **Sharpness-Aware Minimization (SAM):** Tối ưu hóa theo "độ phẳng" của minimaViệc nghiên cứu và phát triển các thuật toán tối ưu mới vẫn đang tiếp diễn, nhằm cải thiện tốc độ hội tụ, độ ổn định, và khả năng tổng quát hóa của các mô hình deep learning.